# Install Libraries

In [ ]:
pip install google-generativeai

# Load Model

In [2]:
import time

In [3]:
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GEMINI_API_KEY")
genai.configure(api_key = api_key)

# Generate content
model = genai.GenerativeModel(model_name='gemini-pro')

In [4]:
def generate_entities (prompt):
    response = model.generate_content(prompt)
    return response.text

In [5]:
def generate_prompt (text,info):
    if info == "CV":
        prompt = '''
        You are a entity extraction expert, you can identify and extract dfferent types of entities from a text. Here is some information from a CV. Your task is to find and enlist all the information entities like education (degree, grade, school name) skills (which skills he has), qualifications (skills), experience (action verb and nouns) and any other helpful token that is important for job, and share them in a list where entities are seperated by comma.  do not write anything else.  Don't share details. just the small entities seperated by comma in a dictonary (json). Each entity can have only 1-2 words..>>'''
        prompt = prompt+"```"+text+"```"
    else:
        prompt = '''
        You are a entity extraction expert, you can identify and extract dfferent types of entities from a text. Here is some information from a job description. Your task is to find and enlist all the information entities like education (degree requirement) skills (which skills the job needs), qualifications (skills), experience (action verb and nouns) and any other helpful token that is important for job, and share them in a list where entities are seperated by comma.  do not write anything else. Don't share details. just the small entities seperated by comma  in a dictonary (json). Each entity can have only 1-2 words.   Don't share details. >>'''
        prompt = prompt+"```"+text+"```"
    prompt = prompt + '''
    =================
    Example:
    {
        'Education': ['ABC University', 'CGPA 3.00', 'Computer Science and Engineering', 'BSc'],
        'Skills' : ['C','Python','R','Machine Learning', 'Communication', 'Team Work']
        'Experience' : {'ABX InfoTech': ['Team Management', 'Assistant Manager'] , 'STech':['Manager','Senior Enginner','AWS']}
    }
    '''
    return prompt

In [6]:
import pandas as pd

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [8]:
JD = pd.read_csv("Selected JDs.csv")
CV = pd.read_csv("Selected CVs.csv")

In [ ]:
outputs_JD = []
outputs_CV = []
for i in range (200):
    JD_i = JD['jobpost'][i]
    CV_i = CV['Resume_str'][i]
    
    print ("Tring: ",i+1)
    
    try:
        JD_t = generate_entities(generate_prompt (JD_i,"JD"))
        print("JD :",JD_t)
    except:
        print("JD Failed")
        
    try:
        CV_t = generate_entities(generate_prompt (CV_i,"CV"))
        print("CV :",CV_t)
    except:
        print("CV Failed")
    
    outputs_JD.append(JD_t)
    outputs_CV.append(CV_t)  
    
    time.sleep(10)

In [10]:
dfx = pd.DataFrame({
    'Outputs_JD': outputs_JD,
    'Outputs_CV': outputs_CV
})

In [11]:
dfx.to_csv("outputs.csv")